In [1]:
import numpy as np
import torch
import torchvision
from torch import Tensor
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import os
import math
import time
import dis
import scipy 

In [3]:
torch.__cached__

'/home/oleksiy-tsuber/miniconda3/lib/python3.13/site-packages/torch/__pycache__/__init__.cpython-313.pyc'

In [4]:
torch.__file__

'/home/oleksiy-tsuber/miniconda3/lib/python3.13/site-packages/torch/__init__.py'

In [2]:
suka = np.random.uniform(0, 1, 10)
ntodo = (10-3)//1 + 1
filter = np.array([0, 0, -1])
filter_long = np.zeros(10)
filter_long[0:3] = filter
filter_long = np.flip(filter_long)

for i in range(0, ntodo):
    print(np.sum(filter*suka[i:i+3]))
print(suka[8]+filter[0]+suka[9]*filter[1]+suka[0]*filter[2])
print(suka[9]*filter[0]+suka[0]*filter[1]+suka[1]*filter[2])
suka_freq = np.fft.rfft(suka)


flongf = np.fft.rfft(filter_long)




fft_ans = np.real(np.fft.irfft(suka_freq*flongf))
print(fft_ans)

-0.4702432204648528
-0.18615768414660772
-0.14434802005511882
-0.2848316590859902
-0.44047283581493235
-0.24504131128749496
-0.83259973089369
-0.9823462874648468
0.3577446888293654
-0.3466969471865544
[-0.18615768 -0.14434802 -0.28483166 -0.44047284 -0.24504131 -0.83259973
 -0.98234629 -0.47485504 -0.34669695 -0.47024322]


In [29]:
def calc_num_computations_of_conv1d(l, kernel_size, stride):
    lout = (l-kernel_size)//stride + 1
    n_calc_conv = lout*kernel_size
    n_calc_fft = l*math.log2(l) + l/2 + l
    print(f"conv1d num ops =     {n_calc_conv}")
    print(f"conv1d fft num ops = {int(n_calc_fft)}")

def calc_num_computations_of_conv2d(in_channels, out_channels, H, W, kernel_size, stride):
    Hout = (H-kernel_size)//stride + 1
    Wout = (W-kernel_size)//stride + 1
    n_calc_regular_conv = Hout*Wout*in_channels*kernel_size*kernel_size*out_channels
    n_calc_depth_sep_conv = Hout*Wout*in_channels*kernel_size*kernel_size + out_channels*in_channels*Hout*Wout
    n_calc_regular_conv_fft = in_channels*(0.25*H*W*math.log2(H*W)) +  H*W*in_channels*out_channels + out_channels*(0.25*Hout*Wout*math.log2(Hout*Wout))
    n_calc_depth_sep_conv_fft = in_channels*(0.25*H*W*math.log2(H*W))  + H*W*in_channels*out_channels + out_channels*(0.25*Hout*Wout*math.log2(Hout*Wout))
    print(f"n ops regular conv =       {n_calc_regular_conv}")
    print(f"n ops depth sep conv =     {n_calc_depth_sep_conv}")
    print(f"n ops regular conv fft =   {int(n_calc_regular_conv_fft)}")
    print(f"n ops depth sep conv fft = {int(n_calc_depth_sep_conv_fft)}")
calc_num_computations_of_conv1d(100000, 3, 1)
print("")
calc_num_computations_of_conv2d(64, 64, 640, 640, 3, 1)

conv1d num ops =     299994
conv1d fft num ops = 1810964

n ops regular conv =       15005270016
n ops depth sep conv =     1901709568
n ops regular conv fft =   1921269076
n ops depth sep conv fft = 1921269076


In [10]:
def one_test(in_channels, out_channels, H, W, kernel_size, stride, padding, numthreads):
    inp = torch.randint(-256, 257, (1, in_channels, W, H), dtype=torch.float32)
    we = torch.randint(-1, 2, (out_channels, in_channels, kernel_size, kernel_size), dtype=torch.float32)
    inp_padded = torch.zeros((1, in_channels, H+2*padding, W+2*padding), dtype=torch.float32)
    inp_padded[..., padding:padding+H, padding:padding+W] = inp
    st = time.time()
    outp = torch.nn.functional.conv2d(inp_padded, we, None, stride, 0, 1, 1)
    end = time.time()
    print(f"pytorch conv2d took {end - st} s")
    if out_channels > 1:
        outp = torch.squeeze(outp, 0)
    inp_numpy = inp_padded.numpy()
    we_numpy = we.numpy()
    outp_numpy = outp.numpy()

    np.savetxt("input.txt", inp_numpy.flatten(), "%d")
    np.savetxt("weight.txt", we_numpy.flatten(), "%d")
    np.savetxt("output.txt", outp_numpy.flatten(), "%d")
    with open("conv_params.txt", "w") as f:
        f.write(f'{numthreads} {out_channels} {in_channels} {H} {W} {kernel_size} {stride} {padding}')
    
    if os.path.exists('regular_conv_with_reshaping'):
        os.remove('regular_conv_with_reshaping')
    !gcc -march=native -O3 -g -o regular_conv_with_reshaping regular_conv_with_reshaping.c -lblas
    !time ./regular_conv_with_reshaping

In [11]:
one_test(64, 64, 640, 640, 3, 1, 1, 1)

pytorch conv2d took 0.0969693660736084 s
regular_conv_with_reshaping.c: In function ‘main’:
regular_conv_with_reshaping.c:144:5: warning: ignoring return value of ‘fscanf’ declared with attribute ‘warn_unused_result’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-result-Wunused-result]8;;]
  144 |     fscanf(f, "%d ", &NUMTHREADS);
      |     ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~
regular_conv_with_reshaping.c:145:5: warning: ignoring return value of ‘fscanf’ declared with attribute ‘warn_unused_result’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-result-Wunused-result]8;;]
  145 |     fscanf(f, "%d ", &out_channels);
      |     ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
regular_conv_with_reshaping.c:146:5: warning: ignoring return value of ‘fscanf’ declared with attribute ‘warn_unused_result’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-result-Wunused-result]8;;]
  146 |     fscanf(f, "%d ", &D);
      |